In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [2]:
# generating dataset for discriminator
def get_dataset():
    return torch.Tensor(np.random.normal(4,1.25,(1,100)))

In [3]:
# generating noise for generator
def make_noise():
    return torch.Tensor(np.random.uniform(0,0,(1,50)))

In [4]:
class generator(nn.Module):
    
    def __init__(self,inp,out):
        super(generator,self).__init__()
        self.net = nn.Sequential(nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                nn.Linear(300,out)
                               )
        
    def forward(self,x):
        x = self.net(x)
        return x
        

In [5]:
class discriminator(nn.Module):
    
    def __init__(self,inp,out):
        super(discriminator,self).__init__()
        self.net = nn.Sequential(nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,out),
                                 nn.Sigmoid()
                                )
        
    def forward(self,x):
        x = self.net(x)
        return x

In [6]:
def stats(array):
    array = array.detach().numpy()
    return [np.mean(array),np.std(array)]

In [7]:
gen = generator(50,100)

In [8]:
gen

generator(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=300, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=300, out_features=100, bias=True)
  )
)

In [9]:
x = make_noise()

In [10]:
x.shape

torch.Size([1, 50])

In [12]:
x1 = gen(x)

In [14]:
x1.shape

torch.Size([1, 100])

In [17]:
stats(x1)

[0.0016335561, 0.039144013]

In [18]:
discrim = discriminator(100,1)

In [19]:
discrim

discriminator(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=300, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=300, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [20]:
x2 = discrim(x1)

In [21]:
x2

tensor([[0.4936]], grad_fn=<SigmoidBackward0>)

In [22]:
epochs = 500

In [23]:

d_step = 10
g_step = 8
criteriond1 = nn.BCELoss()
optimizerd1 = optim.SGD(discrim.parameters(), lr=0.001, momentum=0.9)

criteriond2 = nn.BCELoss()
optimizerd2 = optim.SGD(gen.parameters(), lr=0.001, momentum=0.9)

printing_steps = 20

In [24]:
torch.ones(1,1)

tensor([[1.]])

In [25]:
for epoch in range(epochs):
    ## 核心：每一个epoch里面，生成器训练一下，变强，然后再轮给辨别器训练一下，也变强。
    ## 最后生成器和辨别器都变得很强了。
    ## 这里面，假图和真图的标签是直接用torch.zeros, torch.ones生成的，也就是钦定的。
    ## 嗨，是不是，你觉得很难的算法，其实掰开了都不会很难，或者说核心都是简单的。
    
    if epoch%printing_steps==0:
        print("Epoch:", epoch)
    
    # training discriminator
    for d_i in range(d_step): 
        ## 辨别器先单独训练10轮，然后生成器在单独训练8轮。
        ## 辨别器的目标就是，读取一张真实图片，把它判断为真图。
        ## 随即读取一张假图，将之判断为假图。
        ## 也就是说，辨别器需要把图的真实情况辨别出来，所以既要懂得真货长啥样，也要懂得假货长啥样。
        discrim.zero_grad()
        
        #real
        data_d_real = Variable(get_dataset())
        data_d_real_pred = discrim(data_d_real)
        data_d_real_loss = criteriond1(data_d_real_pred,Variable(torch.ones(1,1)))
        data_d_real_loss.backward()
        
        
        #fake
        data_d_noise = Variable(make_noise())
        data_d_gen_out = gen(data_d_noise).detach() ## 这里，gen的运算不纳入到梯度计算当中，应当只在gen的训练过程中纳入梯度计算。
        data_fake_dicrim_out = discrim(data_d_gen_out)
        data_fake_d_loss = criteriond1(data_fake_dicrim_out,Variable(torch.zeros(1,1)))
        data_fake_d_loss.backward()
        
        optimizerd1.step()
        
    for g_i in range(g_step): 
        ## 喏，生成器来了。这里自己先跑8轮，每一轮只生成一张图。
        ## 生成一个假图，然后这个图被辨别器判断为真图。
        ## 整个过程中，要更新生成器的参数。
        ## 通俗话讲，就是假货根据自己的感觉，把图生成成自己觉得像真货的样子。
        gen.zero_grad()
        
        data_noise_gen = Variable(make_noise())
        data_g_gen_out = gen(data_noise_gen)
        data_g_dis_out = discrim(data_g_gen_out)
        data_g_loss = criteriond2(data_g_dis_out,Variable(torch.ones(1,1)))
        data_g_loss.backward()
        
        optimizerd2.step()
        
        if epoch%printing_steps==0:
            print(stats(data_g_gen_out))
    
    if epoch%printing_steps==0:
        print("\n\n")
    

Epoch: 0
[0.0016335561, 0.039144013]
[0.0016359552, 0.039144386]
[0.0016405022, 0.03914513]
[0.001646986, 0.0391462]
[0.0016552131, 0.03914757]
[0.0016650116, 0.039149236]
[0.0016762214, 0.039151173]
[0.0016887039, 0.03915337]



Epoch: 20
[0.010587318, 0.03810849]
[0.010663982, 0.03809866]
[0.010740876, 0.038089383]
[0.010817978, 0.03808085]
[0.010895262, 0.038073123]
[0.010972977, 0.038066484]
[0.011051335, 0.038061216]
[0.011130328, 0.03805738]



Epoch: 40
[0.03368272, 0.06299022]
[0.03390099, 0.063307285]
[0.03411659, 0.06362578]
[0.03433002, 0.06394556]
[0.034546394, 0.06426869]
[0.034766678, 0.064595155]
[0.034991134, 0.06492497]
[0.03521941, 0.065258205]



Epoch: 60
[0.09592845, 0.14154112]
[0.09665898, 0.14217396]
[0.0973979, 0.14280893]
[0.09814462, 0.14344665]
[0.098898135, 0.14408834]
[0.0996585, 0.14473371]
[0.100426465, 0.14538291]
[0.10120194, 0.14603662]



Epoch: 80
[0.33868617, 0.2939974]
[0.34165776, 0.2951768]
[0.34464785, 0.29634568]
[0.34765995, 0.29750937]
[0.35

In [26]:
gen

generator(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=300, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=300, out_features=100, bias=True)
  )
)

In [20]:
# torch.save(gen.state_dict(),"d_step_5000_g_step_2000")

In [27]:
y = gen(make_noise())

In [28]:
stats(y)

[4.2606764, 0.4248111]